# Taller de procesamiento de BigData en Spark + R
Manuel Parra (manuelparra@decsai.ugr.es). <a href="http://sci2s.ugr.es/es">Soft Computing and Intelligent Information Systems</a>
. <a href="http://sci2s.ugr.es/dicits/">Distributed Computational Intelligence and Time Series</a>. **University of Granada**.
![logos](https://sites.google.com/site/manuparra/home/header.png)

# Operaciones sobre SparkDataFrames:  ``pipes`` y ``magrittr``


Como siempre para todos nuestros `scripts` con **SparkR**, cargamos la biblioteca, y creamos una nueva sesión de SparkR.

En este caso:

<span style="background-color:red;color:white">&nbsp; &nbsp; Cuidado con la cantidad de MEMORIA que usamos para esta sección ! &nbsp; &nbsp; </span>

In [ ]:
#Fijamos la ruta donde está instalado Spark
Sys.setenv("SPARK_HOME"='/usr/local/spark/')

.libPaths(c(file.path(Sys.getenv("SPARK_HOME"),"R/lib/"),.libPaths()))
library(SparkR)
sparkR.session(appName="EntornoInicio", master = "local[*]", sparkConfig = list(spark.driver.memory = "1g"))

Los ``SparkDataFrames`` soportan un alto número de funciones para hacer un procesado de datos estructurado. 

Vamos a poner en práctica las más utilizadas. 

**La lista completa de operaciones que se pueden aplicar se puede ver desde API de SparkR en https://spark.apache.org/docs/latest/api/R/index.html **

![funcSparkR](https://sites.google.com/site/manuparra/home/functionSparkR.jpg)


# Operaciones con SparkDataFrames

In [ ]:
# Primero cargamos los datos
df_nyctrips <- read.df("/SparkR/datasets/yellow_tripdata_2016-02_small1.csv", "csv", header = "true", inferSchema = "true")

In [ ]:
# Hacemos una copia del SparkDataFrame para usarla en una vista temporal en SQL
createOrReplaceTempView(df_nyctrips,"slqdf_filtered_nyc")

# Hacemos una selección de los registros, donde calculamos el tiempo del viaje de cada viaje
sql_nyc <- sql("select VendorID,INT(unix_timestamp(tpep_dropoff_datetime)- unix_timestamp(tpep_pickup_datetime)) AS trip_time,passenger_count,trip_distance,total_amount from slqdf_filtered_nyc")

head(sql_nyc)


## Uso de magrittr para el trabajo con los datos

El paquete ``magrittr`` permite: 

* mejorar el tiempo de desarrollo y 
* mejorar enormemente la legibilidad y mantenibilidad del código. 

Para usarlo hay que importar la biblioteca ``magrittr`` dentro del proyecto y apartir de ese momentos podemos utilizar el operador 

```
%>%
``` 

para concaternar operaciones y poder trabajar con flujos de datos y pipelines.

Provee de un operador que sirve para hacer `pipes` con el cual se puede `encauzar` un valor hacia adelante dentro de una expresión o llamada a función.

Veamos todas las operaciones que hemos realizado sobre los datos y su equivalente con `pipes`.

In [ ]:
# Usamos magrittr
library(magrittr)

# results <- sql("select VendorID, MAX(trip_distance) from slqdf_filtered_nyc GROUP BY VendorID ")
#summarize(groupBy(df_nyctrips, df_nyctrips$passenger_count), count = n(df_nyctrips$passenger_count))

df_nyctrips %>% 
        groupBy( df_nyctrips$passenger_count) %>%
        summarize(count = n(df_nyctrips$passenger_count)) %>%
        head()

In [ ]:
df_nyctrips %>% 
        groupBy( df_nyctrips$passenger_count) %>%
        summarize( avg_total_amount=avg(df_nyctrips$total_amount) ,avg_trip_distance=avg(df_nyctrips$trip_distance)) %>%
        head()
        

<HR>
<div style="font-family:helvetica;padding:5px;font-size:1.5em;background-color:#CFE7E2">Ejercicio práctico:</div>

¿Cómo sería la sentencia del bloque superior en SparkSQL?

<HR>



In [ ]:
df_nyctrips %>% 
         groupBy( df_nyctrips$passenger_count) %>%
        summarize(min = min(df_nyctrips$trip_distance),max = max(df_nyctrips$trip_distance)) %>%
        head()

In [ ]:
df_nyctrips %>% 
         groupBy( df_nyctrips$passenger_count, hour(df_nyctrips$tpep_pickup_datetime)) %>%
        summarize(total_pickup = n(df_nyctrips$tpep_pickup_datetime)) %>%
        head()

In [ ]:
count(df_nyctrips)

num_regs <- as.integer(count(df_nyctrips))

# Mostramos el número de registros
print(num_regs)


<HR>
<div style="font-family:helvetica;padding:5px;font-size:1.5em;background-color:#CFE7E2">Pregunta:</div>

Haz un flujo de funciones con ``magrittr`` donde selecciones las columnas ``total_amount`` y ``passenger_count``, y filtres cuando ``total_amount`` sea mayor de ``50.0``

<HR>


In [ ]:
# El quivalente al codigo del bloque anterior

sql_nyc %>% count()

num_regs <- sql_nyc %>% count() %>% as.integer()

num_regs %>% print()


<HR>
<div style="font-family:helvetica;padding:5px;font-size:1.5em;background-color:#CFE7E2">Pregunta:</div>

¿Cuál es la mejor opción para trabajar: ``pipes``, ``SparkSQL`` o funciones?

<HR>
